<font size=5 color="black">**任務流程**</font>
1. 前置作業 ＝> import 需要套件、建立相關身份認證資料
2. 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面（詳情：selenium-dcard應用），104為動態網頁，使用selenium進行頁面滑動，依個人需求取得各職缺網址，並存成一個網址list
4. 透過迴圈進入各職缺網址內，取得相關資料，append進入output_list中，後續將此list轉成df
5. 將爬蟲回來的資料存成df，進行data cleaning，完成後輸出為excel檔！

參考資料:
<br>
1. [104人力銀行 網路爬蟲](https://tlyu0419.github.io/2020/06/19/Crawler-104HumanResource/)
<br>
2. [[網頁爬蟲] 104人力銀行標籤抓不到內容](https://ithelp.ithome.com.tw/questions/10198403)
<br>
3. [Python 使用 Beautiful Soup 抓取與解析網頁資料，開發網路爬蟲教學](https://blog.gtwang.org/programming/python-beautiful-soup-module-scrape-web-pages-tutorial/2/)

### 前置作業 ＝> import 需要套件、建立headers&相關cookies資料
1. 設置optins => 擋掉跳出視窗
2. 建立headers
3. 準備參數cookies (全職、關鍵字、清單條例式)

In [3]:
import time
import re
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent


In [4]:
# 參數設定
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--headless")

url_search = "https://104.com.tw/jobs/search/?"
ua = UserAgent()
headers = {'User-Agent': ua.google}
my_params = {'ro': '1',                # 限定全職的工作，如果不限定則輸入0
             'keyword': '數據分析',     # 想要查詢的關鍵字
             'mode': 'l'}              # 清單瀏覽模式


### 打開開發人員工具，找到搜尋html，使用selenium輸入關鍵字搜尋，並進入搜尋結果頁面

1. session連線 => 帶著my_params通過搜尋頁面，抵達搜尋結果頁面，104為動態網頁，使用selenium進行頁面滑動
2. 輸入欲爬取至第幾頁面
3. 建立url的beautiful soup物件，取得各職缺網址並存成work_url_lists
    - 使用**driver.page_source**來取得所有頁面資訊
    - 過濾掉置頂徵才廣告 => url中jobsource參數等於hotjob_chr者
    - 取得url中的職缺編號，並組成app版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)

In [5]:
# session 連線
# 帶著參數通過搜尋頁面，抵達結果頁面的網址
ss = requests.session()
res = ss.get(url_search, headers=headers, params=my_params)

# 104為動態網頁，使用selenium進行頁面滑動，蒐集網址
# 輸入欲爬取至第幾頁面
page_number = int(input("請輸入欲爬取至第幾頁面: "))
driver = Chrome("../chromedriver", options=options)
driver.get(res.url)
for i in range(1, page_number+1):
    driver.execute_script("var s = document.documentElement.scrollTop=5000")
    print("目前爬至第{}頁".format(i))
    time.sleep(random.randint(4, 10))

# 建立beautiful soup物件
# 使用driver.page_source來取得所有頁面資訊
# 過濾掉置頂徵才廣告 => url中包含hotjob_chr參數
# 取得url中的職缺編號，並組成手機版網址 =>　https://m.104.com.tw/job/ + 職缺編號(ex. 5944x)
soup = BeautifulSoup(driver.page_source, features='html.parser')
url_data = [url["href"] for url in soup.select('li.job-mode__jobname a') if url["href"].split("=")[1] != 'hotjob_chr']
work_url_lists = list(map(lambda url: 'https://m.104.com.tw/job/{}'.format(re.findall("job/(.*)\?", url)[0]), url_data))


# 關閉頁面
driver.close()

目前爬至第1頁
目前爬至第2頁
目前爬至第3頁
目前爬至第4頁
目前爬至第5頁
目前爬至第6頁
目前爬至第7頁
目前爬至第8頁
目前爬至第9頁
目前爬至第10頁
目前爬至第11頁
目前爬至第12頁
目前爬至第13頁
目前爬至第14頁
目前爬至第15頁
目前爬至第16頁
目前爬至第17頁
目前爬至第18頁
目前爬至第19頁
目前爬至第20頁


In [6]:
print(work_url_lists)

['https://m.104.com.tw/job/799p1', 'https://m.104.com.tw/job/5944x', 'https://m.104.com.tw/job/498rd', 'https://m.104.com.tw/job/6j51q', 'https://m.104.com.tw/job/6sxgk', 'https://m.104.com.tw/job/758eh', 'https://m.104.com.tw/job/7bvz9', 'https://m.104.com.tw/job/6o8n5', 'https://m.104.com.tw/job/7cp7j', 'https://m.104.com.tw/job/7b6ym', 'https://m.104.com.tw/job/7dgi7', 'https://m.104.com.tw/job/71rrj', 'https://m.104.com.tw/job/6jbr5', 'https://m.104.com.tw/job/777rm', 'https://m.104.com.tw/job/7bk2k', 'https://m.104.com.tw/job/6l0xo', 'https://m.104.com.tw/job/7874u', 'https://m.104.com.tw/job/6lg8y', 'https://m.104.com.tw/job/6vpzg', 'https://m.104.com.tw/job/6xtpc', 'https://m.104.com.tw/job/6fjk7', 'https://m.104.com.tw/job/7a385', 'https://m.104.com.tw/job/6hmzf', 'https://m.104.com.tw/job/7b0oa', 'https://m.104.com.tw/job/6x5r9', 'https://m.104.com.tw/job/7bgeg', 'https://m.104.com.tw/job/6bcn9', 'https://m.104.com.tw/job/79ypd', 'https://m.104.com.tw/job/6kzbe', 'https://m.10

### 透過迴圈進入各職缺網址內，取得相關資料，append進入output中，後續將此轉成df
1. 每3秒爬一次，避免被鎖ip或是重導到電腦版網址，因而產生錯誤
2. 取得職缺相關資料
    - 更新時間、公司、職位、工作地點、工作內容
    - 工作需求技能、工具、其他條件
    - 額外資訊(聯絡人、薪水)
5. append進入output

In [7]:
# 每3秒爬一次，避免被鎖或是導向網頁，造成錯誤發生
output = []
for work_url in work_url_lists:
    print(work_url)
    res_work = ss.get(work_url, headers=headers, allow_redirects=False)
    w_soup = BeautifulSoup(res_work.text, 'html.parser')
    
    try:
        # 更新時間、公司、職位、工作地點、工作內容
        update_time = w_soup.select("time")[0]['datetime']
        company = w_soup.select('h2.company')[0].text
        title = w_soup.select('h1.title')[0].text
        work_place = w_soup.select('div.content a.addr')[0].text.strip()
        description = w_soup.select("div.content p")[0].text.strip()
        
        # 工作需求技能、工具、其他條件
        job_skill = " ".join([i.text for i in w_soup.select("a[data-gtm='job-skill']")])
        job_tool = " ".join([i.text for i in w_soup.select("a[data-gtm='job-tool']")])
        if w_soup.select("td div.cut"):
            plus = w_soup.select("td div.cut")[0].text
        else:
            plus = ""
        
        ### 額外資訊(聯絡人、薪水)
        recruiter = w_soup.find_all("table", class_="column2 contact")[0].select("td")[0].text
        extra_data = ''.join([t.text for t in w_soup.select("table.column2 td")])
        salary = re.findall(r"待遇面議.*|[年月]薪.*[元以上]", extra_data) 
        
    except ValueError as e:
        print("Value error", work_url)
    except IndexError as e:
        print("Index error", work_url)
        
    output.append([update_time, company, title, work_place, salary, description, job_skill, job_tool, plus, recruiter, work_url])
    
    time.sleep(3)


https://m.104.com.tw/job/799p1
https://m.104.com.tw/job/5944x
https://m.104.com.tw/job/498rd
https://m.104.com.tw/job/6j51q
https://m.104.com.tw/job/6sxgk
https://m.104.com.tw/job/758eh
https://m.104.com.tw/job/7bvz9
https://m.104.com.tw/job/6o8n5
https://m.104.com.tw/job/7cp7j
https://m.104.com.tw/job/7b6ym
https://m.104.com.tw/job/7dgi7
https://m.104.com.tw/job/71rrj
https://m.104.com.tw/job/6jbr5
https://m.104.com.tw/job/777rm
https://m.104.com.tw/job/7bk2k
https://m.104.com.tw/job/6l0xo
https://m.104.com.tw/job/7874u
https://m.104.com.tw/job/6lg8y
https://m.104.com.tw/job/6vpzg
https://m.104.com.tw/job/6xtpc
https://m.104.com.tw/job/6fjk7
https://m.104.com.tw/job/7a385
https://m.104.com.tw/job/6hmzf
https://m.104.com.tw/job/7b0oa
https://m.104.com.tw/job/6x5r9
https://m.104.com.tw/job/7bgeg
https://m.104.com.tw/job/6bcn9
https://m.104.com.tw/job/79ypd
https://m.104.com.tw/job/6kzbe
https://m.104.com.tw/job/6z47p
https://m.104.com.tw/job/79x3h
https://m.104.com.tw/job/6z9c7
https://

### 將爬蟲回來的資料存成df，進行data cleaning
1. 設置欄位
2. 去除重複值 => 透過company, title, work_place來篩選，保留第一個(日期較新)
3. 建立將df備份成df2，避免資料處理中不小心改動到原檔因而得重新來過

In [8]:
df = pd.DataFrame(output, 
                  columns=['update_time', 'company', 'title', 'work_place', 'salary', 'description', 'job_skill', 'job_tool', "plus", "recruiter", "URL"])

In [9]:
df.shape

(150, 11)

In [41]:
df2 = df.copy()

#### 去除重複值 => 透過company, title, work_place來篩選掉相同重複值缺，保留第一個(日期較新)

In [43]:
df2.drop_duplicates(subset=['company', 'title', 'work_place'], keep="first", inplace=True)
df2.sort_values(by="update_time", ascending=False, inplace=True)
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL
93,2021-09-06,慧邦科技股份有限公司,BI 遊戲數據分析工程師(Data Scientist) -視訊面試,台北市中正區,[待遇面議 ...,【職務說明】1. 數據後台功能開發。2. 數據資料庫建置與維護。3. 報表排程建置與維護。4...,,PHP MySQL jQuery,\n ★因配合防疫措施，目前皆採用視訊面試進行...,余小姐,https://m.104.com.tw/job/6u7bu
9,2021-09-06,芮陽國際有限公司,數據分析師,台中市西屯區潮洋里市政北二路238號15樓之5,[待遇面議 ...,1.具備資料庫結構(MySQL)與擷取資料2.熟悉大數據處理相關技術與統計之資料相關實務經驗...,,Python R MySQL Tableau,\n 需擅長:數據統計分析/機率計算/資料庫應...,芮陽,https://m.104.com.tw/job/7b6ym
22,2021-09-06,神揚保險代理人股份有限公司,數據分析人員,新北市新店區中正路531號6樓,[待遇面議 ...,1.整理編寫每週業務數據分析報告。2.專案需求分析、數據分析、商業分析和數據採擷模型等。3....,,Excel PowerPoint Word 中文打字20~50,\n 1.\t擅溝通、細心、認真負責\r2.\...,徐小姐,https://m.104.com.tw/job/6hmzf
91,2021-09-06,醫療財團法人徐元智先生醫藥基金會亞東紀念醫院,[行政]企劃處/數據分析專員,新北市板橋區南雅南路二段21號 ...,"[月薪 40,000~42,000元]",1. 專案企劃與執行\r2. 資料庫建立與數據分析\r3. 其他交辦事項\r4. 數...,統計軟體操作 文書處理軟體操作,Python R ANSI SQL MS SQL PL/SQL SAS,\n 【科系要求】\r公共衛生、資訊相關、統計...,吳小姐,https://m.104.com.tw/job/6ds2z
88,2021-09-06,愛達科技有限公司,網路行銷數據分析人員,台北市松山區復興北路231巷19-1號2樓(仁富框廠後棟2樓) ...,"[月薪 33,000~38,000元]",1.官網及各站台momo、雅虎、蝦皮等行銷活动規劃。\r2.具美感並能與設計溝通版面構成，規...,,Google Analytics Google Trend,,康先生,https://m.104.com.tw/job/6el89
...,...,...,...,...,...,...,...,...,...,...,...
139,2021-07-12,中山醫學大學附設醫院,臨床數據分析師,台中市南區建國北路一段110號 ...,"[月薪 34,000元以上]",1.臨床研究需求與可行性評估2.臨床資料銜接3.臨床資料清潔整理與統計分析4.臨床影像標註5...,,,\n 有資料庫實務經驗擅長統計分析臨床醫護經驗...,黃小姐,https://m.104.com.tw/job/77va2
81,2021-06-25,DECATHLON法商迪卡儂_台灣迪卡儂有限公司,[電子商務部] 數據分析師 Data Analyst,台中市南屯區大墩南路379號 ...,[待遇面議 ...,1.根據數據發現業務提升點、發揮數據價值導向，深入了解各平台廣告投放規則，給予有效的運營類數...,專案時間╱進度控管 專案溝通╱整合管理 專案管理架構及專案說明 提案與簡報技巧 網路活動規劃...,PHP Python MySQL JavaScript XML jQuery Tableau...,"\n -熱愛運動, 經常從事至少一項以上的運動...",Jack,https://m.104.com.tw/job/76sjb
104,2021-05-05,聯合報股份有限公司,【數據發展部】助理數據分析師,新北市汐止區大同路一段369號,"[月薪 34,000~45,000元]",1.\t擔任精準廣告溝通橋樑，負責監控、優化、管理專案進度。\r2.\t定期分析廣告/EDM...,,Python Excel SPSS,\n 1.\t有一年以上的廣告代理商經驗佳。\...,黃小姐,https://m.104.com.tw/job/70mim
101,2021-05-05,聯合報股份有限公司,【數據發展部】數據分析師,新北市汐止區大同路一段369號,"[月薪 38,000~58,000元]",本職務主要是透過各種客戶行為資料，分析客戶行為特徵，找出客戶行為模式，即時及預測客戶的需要，...,,PL/SQL Excel,\n 一、個人特質：\r(一)邏輯思考、分析判...,黃小姐,https://m.104.com.tw/job/4u5lb


#### 薪資(salary)
將薪資進行統一格式處理
- 分成下限 & 上限 => 以利後續查找資料
- 年薪改為月薪 => 除以12
- 待遇面議改為下限4w、上限面議



In [44]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(" ", "", x[0]))

In [45]:
df2.loc[(df2["salary"].str.contains("待遇面議")), "salary"] = "40000~0"

In [46]:
df2["salary"] = df2["salary"].apply(lambda x: re.sub(",|[^0-9~]", "", x)) 

In [47]:
# 有些只寫起薪的會造成error，index用-1避免報錯
df2["salary_min"] = df2["salary"].map(lambda x: x.split("~")[0])
df2["salary_max"] = df2["salary"].map(lambda x: x.split("~")[-1])

In [48]:
# data type轉成 numeric
df2[["salary_min", "salary_max"]] = df2[["salary_min", "salary_max"]].apply(pd.to_numeric)

In [49]:
## 年薪改為月薪 => 此處用平均數加上3倍標準差來過濾出極端值後進行處理，應該有更好的方法
cond_low = df2["salary_min"].mean() + 3*df2["salary_min"].std()
cond_upper = df2["salary_max"].mean() + 3*df2["salary_max"].std()

df2["salary_min"] = df2["salary_min"].apply(lambda x: int(x/12) if x > cond_low else x)
df2["salary_max"] = df2["salary_max"].apply(lambda x:int(x/12) if x >cond_upper else x)

In [50]:
# 只寫起薪或salary_max等於0者，設定為"面議"
df2.loc[(df2["salary_min"] == df2["salary_max"]) | (df2["salary_max"] == 0), "salary_max"] = "面議"

In [51]:
df2

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL,salary_min,salary_max
93,2021-09-06,慧邦科技股份有限公司,BI 遊戲數據分析工程師(Data Scientist) -視訊面試,台北市中正區,40000~0,【職務說明】1. 數據後台功能開發。2. 數據資料庫建置與維護。3. 報表排程建置與維護。4...,,PHP MySQL jQuery,\n ★因配合防疫措施，目前皆採用視訊面試進行...,余小姐,https://m.104.com.tw/job/6u7bu,40000,面議
9,2021-09-06,芮陽國際有限公司,數據分析師,台中市西屯區潮洋里市政北二路238號15樓之5,40000~0,1.具備資料庫結構(MySQL)與擷取資料2.熟悉大數據處理相關技術與統計之資料相關實務經驗...,,Python R MySQL Tableau,\n 需擅長:數據統計分析/機率計算/資料庫應...,芮陽,https://m.104.com.tw/job/7b6ym,40000,面議
22,2021-09-06,神揚保險代理人股份有限公司,數據分析人員,新北市新店區中正路531號6樓,40000~0,1.整理編寫每週業務數據分析報告。2.專案需求分析、數據分析、商業分析和數據採擷模型等。3....,,Excel PowerPoint Word 中文打字20~50,\n 1.\t擅溝通、細心、認真負責\r2.\...,徐小姐,https://m.104.com.tw/job/6hmzf,40000,面議
91,2021-09-06,醫療財團法人徐元智先生醫藥基金會亞東紀念醫院,[行政]企劃處/數據分析專員,新北市板橋區南雅南路二段21號 ...,40000~42000,1. 專案企劃與執行\r2. 資料庫建立與數據分析\r3. 其他交辦事項\r4. 數...,統計軟體操作 文書處理軟體操作,Python R ANSI SQL MS SQL PL/SQL SAS,\n 【科系要求】\r公共衛生、資訊相關、統計...,吳小姐,https://m.104.com.tw/job/6ds2z,40000,42000
88,2021-09-06,愛達科技有限公司,網路行銷數據分析人員,台北市松山區復興北路231巷19-1號2樓(仁富框廠後棟2樓) ...,33000~38000,1.官網及各站台momo、雅虎、蝦皮等行銷活动規劃。\r2.具美感並能與設計溝通版面構成，規...,,Google Analytics Google Trend,,康先生,https://m.104.com.tw/job/6el89,33000,38000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2021-07-12,中山醫學大學附設醫院,臨床數據分析師,台中市南區建國北路一段110號 ...,34000,1.臨床研究需求與可行性評估2.臨床資料銜接3.臨床資料清潔整理與統計分析4.臨床影像標註5...,,,\n 有資料庫實務經驗擅長統計分析臨床醫護經驗...,黃小姐,https://m.104.com.tw/job/77va2,34000,面議
81,2021-06-25,DECATHLON法商迪卡儂_台灣迪卡儂有限公司,[電子商務部] 數據分析師 Data Analyst,台中市南屯區大墩南路379號 ...,40000~0,1.根據數據發現業務提升點、發揮數據價值導向，深入了解各平台廣告投放規則，給予有效的運營類數...,專案時間╱進度控管 專案溝通╱整合管理 專案管理架構及專案說明 提案與簡報技巧 網路活動規劃...,PHP Python MySQL JavaScript XML jQuery Tableau...,"\n -熱愛運動, 經常從事至少一項以上的運動...",Jack,https://m.104.com.tw/job/76sjb,40000,面議
104,2021-05-05,聯合報股份有限公司,【數據發展部】助理數據分析師,新北市汐止區大同路一段369號,34000~45000,1.\t擔任精準廣告溝通橋樑，負責監控、優化、管理專案進度。\r2.\t定期分析廣告/EDM...,,Python Excel SPSS,\n 1.\t有一年以上的廣告代理商經驗佳。\...,黃小姐,https://m.104.com.tw/job/70mim,34000,45000
101,2021-05-05,聯合報股份有限公司,【數據發展部】數據分析師,新北市汐止區大同路一段369號,38000~58000,本職務主要是透過各種客戶行為資料，分析客戶行為特徵，找出客戶行為模式，即時及預測客戶的需要，...,,PL/SQL Excel,\n 一、個人特質：\r(一)邏輯思考、分析判...,黃小姐,https://m.104.com.tw/job/4u5lb,38000,58000


#### 地區
1. 去除所有空格
2. 從work_place中分成出area

In [52]:
df2["work_place"] = df2["work_place"].apply(lambda x: re.sub(" ", "", x))

In [53]:
df2["area"] = df2["work_place"].str.strip().str[:3]

In [54]:
df2["work_place"] = df2["work_place"].str.strip().str[3:]

#### 處理job_skill 、 job_tool

- 為空值者，設為不拘 


In [55]:
df2.loc[(df2["job_tool"] == "") | (df2["job_skill"] == ""), ["job_skill", "job_tool"]] = "不拘"

#### 其他條件(plus)、工作內容(description) => 除去任何空白字符
- 其他條件(plus)為空白者，設為無

In [56]:
df2["plus"] = df2["plus"].apply(lambda x: re.sub(r"\s", "", x)) ## r"\s" => r""表示此字串套用re規則，\s代表匹配任何空白字符

In [57]:
df2.loc[(df2["plus"] == ""), "plus"] = "無"

In [58]:
df2["description"] = df2["description"].apply(lambda x: re.sub(r"\s", "", x))

In [28]:
df2.head()

,update_time,company,title,work_place,salary,description,job_skill,job_tool,plus,recruiter,URL,salary_min,salary_max,area
93,2021-09-06,慧邦科技股份有限公司,BI 遊戲數據分析工程師(Data Scientist) -視訊面試,中正區,40000~0,【職務說明】1.數據後台功能開發。2.數據資料庫建置與維護。3.報表排程建置與維護。4.使用...,不拘,不拘,★因配合防疫措施，目前皆採用視訊面試進行。,余小姐,https://m.104.com.tw/job/6u7bu,40000,面議,台北市
9,2021-09-06,芮陽國際有限公司,數據分析師,西屯區潮洋里市政北二路238號15樓之5,40000~0,1.具備資料庫結構(MySQL)與擷取資料2.熟悉大數據處理相關技術與統計之資料相關實務經驗...,不拘,不拘,需擅長:數據統計分析/機率計算/資料庫應用,芮陽,https://m.104.com.tw/job/7b6ym,40000,面議,台中市
22,2021-09-06,神揚保險代理人股份有限公司,數據分析人員,新店區中正路531號6樓,40000~0,1.整理編寫每週業務數據分析報告。2.專案需求分析、數據分析、商業分析和數據採擷模型等。3....,不拘,不拘,1.擅溝通、細心、認真負責2.數字邏輯佳，具數據分析相關經驗3.具財產和人身保險業務員資格證...,徐小姐,https://m.104.com.tw/job/6hmzf,40000,面議,新北市
91,2021-09-06,醫療財團法人徐元智先生醫藥基金會亞東紀念醫院,[行政]企劃處/數據分析專員,板橋區南雅南路二段21號（距捷運亞東醫院站140公尺）,40000~42000,1.專案企劃與執行2.資料庫建立與數據分析3.其他交辦事項4.數據中心儲備人才,統計軟體操作 文書處理軟體操作,Python R ANSI SQL MS SQL PL/SQL SAS,【科系要求】公共衛生、資訊相關、統計學、流行病學、醫務管理相關1.國內外碩士，擅長統計分析與...,吳小姐,https://m.104.com.tw/job/6ds2z,40000,42000,新北市
88,2021-09-06,愛達科技有限公司,網路行銷數據分析人員,松山區復興北路231巷19-1號2樓(仁富框廠後棟2樓)（距捷運中山國中站420公尺）,33000~38000,1.官網及各站台momo、雅虎、蝦皮等行銷活动規劃。2.具美感並能與設計溝通版面構成，規劃廣...,不拘,不拘,無,康先生,https://m.104.com.tw/job/6el89,33000,38000,台北市


#### 重新排序columns

- 欄位重新排為以下順序 => 'update_time', 'company', 'title', 'area', 'work_place', 'salary_min', 'salary_max', 'description', 'job_skill', 'job_tool', 'plus', 'recruiter', 'URL'

In [59]:
col = list(df2.columns)
col

['update_time',
 'company',
 'title',
 'work_place',
 'salary',
 'description',
 'job_skill',
 'job_tool',
 'plus',
 'recruiter',
 'URL',
 'salary_min',
 'salary_max',
 'area']

In [60]:
df2 = df2[col[:3] + [col[-1]] + [col[3]] + col[-3:-1] + col[5:-3]]

In [69]:
# 將URL從手機版轉換為網頁版
df2["URL"] = df2["URL"].apply(lambda x: re.sub("(m.)", "", x, 1)) # 只取代第一個match的string

<ipython-input-69-c7727df31257>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["URL"] = df2["URL"].apply(lambda x: re.sub("(m.)", "", x, 1)) # 只取代第一個match的string


#### 重設index

In [70]:
df2.reset_index(drop=True, inplace=True)

In [71]:
df2.head()

,update_time,company,title,area,work_place,salary_min,salary_max,description,job_skill,job_tool,plus,recruiter,URL
0,2021-09-06,慧邦科技股份有限公司,BI 遊戲數據分析工程師(Data Scientist) -視訊面試,台北市,中正區,40000,面議,【職務說明】1.數據後台功能開發。2.數據資料庫建置與維護。3.報表排程建置與維護。4.使用...,不拘,不拘,★因配合防疫措施，目前皆採用視訊面試進行。,余小姐,https://104.com.tw/job/6u7bu
1,2021-09-06,芮陽國際有限公司,數據分析師,台中市,西屯區潮洋里市政北二路238號15樓之5,40000,面議,1.具備資料庫結構(MySQL)與擷取資料2.熟悉大數據處理相關技術與統計之資料相關實務經驗...,不拘,不拘,需擅長:數據統計分析/機率計算/資料庫應用,芮陽,https://104.com.tw/job/7b6ym
2,2021-09-06,神揚保險代理人股份有限公司,數據分析人員,新北市,新店區中正路531號6樓,40000,面議,1.整理編寫每週業務數據分析報告。2.專案需求分析、數據分析、商業分析和數據採擷模型等。3....,不拘,不拘,1.擅溝通、細心、認真負責2.數字邏輯佳，具數據分析相關經驗3.具財產和人身保險業務員資格證...,徐小姐,https://104.com.tw/job/6hmzf
3,2021-09-06,醫療財團法人徐元智先生醫藥基金會亞東紀念醫院,[行政]企劃處/數據分析專員,新北市,板橋區南雅南路二段21號（距捷運亞東醫院站140公尺）,40000,42000,1.專案企劃與執行2.資料庫建立與數據分析3.其他交辦事項4.數據中心儲備人才,統計軟體操作 文書處理軟體操作,Python R ANSI SQL MS SQL PL/SQL SAS,【科系要求】公共衛生、資訊相關、統計學、流行病學、醫務管理相關1.國內外碩士，擅長統計分析與...,吳小姐,https://104.com.tw/job/6ds2z
4,2021-09-06,愛達科技有限公司,網路行銷數據分析人員,台北市,松山區復興北路231巷19-1號2樓(仁富框廠後棟2樓)（距捷運中山國中站420公尺）,33000,38000,1.官網及各站台momo、雅虎、蝦皮等行銷活动規劃。2.具美感並能與設計溝通版面構成，規劃廣...,不拘,不拘,無,康先生,https://104.com.tw/job/6el89


#### 存成excel檔

In [72]:
df2.to_excel("./104_job_bank(data analysis related).xlsx", index=False)